# TP 1 ML

In [ ]:
import gc
import requests
import pandas as pd
import numpy as np
import seaborn as sns
from math import sqrt
from bs4 import BeautifulSoup
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from statsmodels.formula.api import ols
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', None)

## Query Data

Toma aprox 10 minutos en hacer todo el scrape, si definen pages = [1] (linea 4) se hace en 1 minuto!!
La base data.parquet es el resultado de este scrap.

In [ ]:
# dfs = []
# pages = range(1, 11)
# # Toma aprox 10 minutos, si quieren pueden desmutear la linea de abajo para correrlo más rápido
# # pages = [1]

# for page in tqdm(pages):

#   url = f'https://ignaciomsarmiento.github.io/GEIH2018_sample/pages/geih_page_{page}.html'

#   response = requests.get(url)

#   soup = BeautifulSoup(response.content, 'html.parser')

#   table = soup.find('table')

#   data = []

#   # Extract the table headers
#   headers = []
#   for header in table.find_all('th'):
#       headers.append(header.text.strip())
#   data.append(headers)

#   # Extract the table rows
#   for row in table.find_all('tr'):
#       # Extract the table data
#       row_data = []
#       for cell in row.find_all('td'):
#           row_data.append(cell.text.strip())
#       if row_data:
#           data.append(row_data)

#   df_temp = pd.DataFrame(data, columns=headers)
#   dfs += [df_temp]

# df = pd.concat(dfs)
# df.to_parquet("data.parquet")

In [ ]:
df = pd.read_parquet("stores/data.parquet")

In [ ]:
df.info()

In [ ]:
df.sample(10)

# Data cleaning

In [ ]:
# Solo me quedo con las variables que el Estado (sin la encuesta) podría observar para un individuo determinado.
#   - Todas las Transferencias y beneficios estatales están incluídos
#   - Vamos a generar variables "simuladas" donde los trabajadores informales tienen ingreso laboral 0

df = df.rename(columns={"sex":"male"})
df_full = df.copy()
variables_relevantes = [
    "ingtotes", "ingtot", "ingtotob", # Las que queremos predecir: Ingreso total (con y sin imputaciones?)
    "ocu", "formal", # Filtro para ocupados formales (porque solo esos puede observar el Estado)
    "age", "male", "maxEducLevel", "college", "microEmpresa", "sizeFirm", "clase", "cuentaPropia", "relab", # Variables clasicas de Mincer
    "estrato1", "depto", # Efectos fijos por region/estrato
    "regSalud", "cotPension", "y_accidentes_m", "y_auxilioAliment_m", "y_auxilioTransp_m", # Información observable por el Estado que puede ser util!
    "mes" # Para estacionalidad
]

cat_vars = ["maxEducLevel", "relab", "estrato1", "depto", "regSalud", "cotPension"]
df = df[variables_relevantes]

In [ ]:
## Formateo de la base
# Elimino los nombres de las columnas en la primera observacion
df = df.iloc[1:] # (me quedo de la fila 1 en adelante)

## Formateo columnas
# Paso a numericas
df = df.apply(pd.to_numeric, errors="coerce")

# Paso a bool las dummies:
bool_vars = []
print(f"### Transformación a Bool ###")
for col in df.columns:
  if df[col].isin([np.nan, 0,1]).all():
    print(f"Columna {col} pasada a bool dtype")
    df[col] = df[col].astype(bool)
    bool_vars += [col]

#Filtro por edad, ocupación y mayores de edad (FIXME: estamos de acuerdo en trabajar solo con formales?)
df = df[(df["ocu"] == 1) & (df["age"] >= 18) & (df["formal"]==True)]

# Genero binarias de las variables categóricas:
print(f"### Transformación a Categórica ###")
dummies = []
df = pd.get_dummies(df, columns=cat_vars, drop_first=True, dummy_na=True)
df.columns = [col.replace(".0","") for col in df.columns]
for col in cat_vars:
  print(f"Columnas {col} (categórica) transformada a binarias")

# Genero lista con variables continuas
cont_vars = list(set(variables_relevantes) - set(bool_vars) - set(cat_vars))
df.info()

In [ ]:
## Limpio y genero variables
# Las variables de ingresos observables extras tienen muchos nan pero deben ser interpretadas como ceros
df["y_accidentes_m"] = df["y_accidentes_m"].fillna(0)
df["y_auxilioAliment_m"] = df["y_auxilioAliment_m"].fillna(0)
df["y_auxilioTransp_m"] = df["y_auxilioTransp_m"].fillna(0)

# Genero logs "ingtotes", "ingtot", "ingtotob"
df["log_ingtotes"] = np.log(df["ingtotes"])
df["log_ingtotob"] = np.log(df["ingtotob"])
df["log_ingtot"]   = np.log(df["ingtot"])

In [ ]:
## Formateo el dataframe
df = df.iloc[:,(df.nunique() > 1).values] # Elimino columnas constantes
df = df.dropna(how="all", axis=1) # O que tienen todo nan
df = df.reset_index(drop=True)

# Redefino los conjuntos
cont_vars = [col for col in cont_vars if col in df.columns]
bool_vars = [col for col in bool_vars if col in df.columns]
cat_vars  = [col for col in df.columns if any([old_col for old_col in cat_vars if old_col in col])]

cat_vars

In [ ]:
df.sample(10)

## Analisis descriptivas

In [ ]:
import missingno as msno
import matplotlib.pyplot as plt
df_full = df_full[(df_full["ocu"] == "1") & (df_full["age"] >= "18") & (df_full["formal"]=="1")]
fig = msno.matrix(df_full[variables_relevantes].replace("NA",np.nan, regex=True), figsize=(10,6), fontsize=12, labels=True, color=(0.25, 0.45, 0.6))
plt.savefig('views/missingplot.jpg',bbox_inches='tight', dpi=150)

In [ ]:
stats = df[cont_vars].describe().T
stats[["count", "mean", "50%", "std"]].rename(columns={"50%":"median"}).to_latex("views//descriptive_stats.tex")

In [ ]:
sns.set(rc={'figure.figsize':(6,6)})
sns.set_theme(style="ticks")

ax = sns.histplot(df["ingtot"], fill=True)#, bw_adjust=0.15)
plt.xlim([0,3_500_000])
ax.ticklabel_format(style='plain')
ax.set_xlabel("Ingreso total")
ax.set_ylabel("Frecuencia")
plt.savefig('views/ingreso_tot_dist.jpg',bbox_inches='tight', dpi=150)

In [ ]:
ax = sns.scatterplot(data=df, y="log_ingtot", x="log_ingtotob", s=10)#, bw_adjust=0.15)
plt.xlim([10,18])
plt.ylim([10,18])
ax.ticklabel_format(style='plain')
ax.set_xlabel("Ingreso total observado, en logaritmos")
ax.set_ylabel("Ingreso total (observado + imputado), en logaritmos")
sns.despine()
plt.savefig('views/ingreso_tot_vs_obs.jpg',bbox_inches='tight', dpi=150)

In [ ]:
df.describe()

## Estimacion de modelos

In [ ]:
X = df.loc[:,~df.columns.isin(["ingtotes", "ingtot", "ingtotob", "log_ingtotes", "log_ingtot", "log_ingtotob"])]

y = df[["ingtot"]]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
                                        X,
                                        y,
                                        test_size=0.3,
                                        train_size=0.7,
                                        random_state = 84
                                    )

# Predicting wages

In [ ]:
#prediction on new data
X0_test = np.ones((len(y_test), 1))
y_hat_model1 = model1.predict(X0_test)

In [ ]:
vars_model1 = []
x_train_model1 = np.ones((len(y_train), 1))
x_test_model1  = np.ones((len(y_test), 1))

model1 =  LinearRegression().fit(x_train_model1,y_train)

In [ ]:
vars_model2 = ['age']
x_train_model2 = X_train[vars_model2]
x_test_model2  = X_test[vars_model2]
model2=  LinearRegression().fit(x_train_model2,y_train)
model2.coef_

In [ ]:
vars_model3 = [
    'age',
    'male',
    'maxEducLevel_3',
    'maxEducLevel_4',
    'maxEducLevel_5',
    'maxEducLevel_6',
    'maxEducLevel_7',
    "maxEducLevel_nan",
]
x_train_model3 = X_train[vars_model3]
x_test_model3  = X_test[vars_model3]
model3 = LinearRegression().fit(x_train_model3,y_train)
model3.coef_

In [ ]:
vars_model4 = [
    'age',
    'male',
    'maxEducLevel_3',
    'maxEducLevel_4',
    'maxEducLevel_5',
    'maxEducLevel_6',
    'maxEducLevel_7',
    "maxEducLevel_nan",
    'college',
    'microEmpresa',
    'sizeFirm',
    'regSalud_2',
    'cotPension_3',
    'mes',
  ]
x_train_model4 = X_train[vars_model4]
x_test_model4 = X_test[vars_model4]

model4 = LinearRegression().fit(x_train_model4,y_train)
model4.coef_

In [ ]:
X_train.loc[:, ~X_train.columns.isin([
    'age',
    'male',
    'maxEducLevel_3',
    'maxEducLevel_4',
    'maxEducLevel_5',
    'maxEducLevel_6',
    'maxEducLevel_7',
    'college',
    'microEmpresa',
    'sizeFirm',
    'regSalud_2',
    'cotPension_3',
    'mes',
  ])]

In [ ]:
x_train_model5 = X_train
x_test_model5 = X_test

model5 = LinearRegression().fit(X_train,y_train)
model5.coef_

In [ ]:
poly = PolynomialFeatures(degree=2)

x_train_model6 = poly.fit_transform(
    X_train[
      ['age',
      'male',
      'maxEducLevel_3',
      'maxEducLevel_4',
      'maxEducLevel_5',
      'maxEducLevel_6',
      'maxEducLevel_7',
      'maxEducLevel_nan',
      'college',
      'microEmpresa',
      'sizeFirm',
      'regSalud_2',
      'cotPension_3']]
)
x_test_model6 = poly.fit_transform(
    X_test[
      ['age',
      'male',
      'maxEducLevel_3',
      'maxEducLevel_4',
      'maxEducLevel_5',
      'maxEducLevel_6',
      'maxEducLevel_7',
      'maxEducLevel_nan',
      'college',
      'microEmpresa',
      'sizeFirm',
      'regSalud_2',
      'cotPension_3']]
)


model6 =  LinearRegression().fit(x_train_model6,y_train)

In [ ]:
poly = PolynomialFeatures(degree=2)

x_train_model7 = poly.fit_transform(X_train)
x_test_model7  = poly.fit_transform(X_test)
model7 =  LinearRegression().fit(x_train_model7,y_train)

In [ ]:
poly = PolynomialFeatures(degree=3)

x_train_model8 = poly.fit_transform(X_train[['age',
                                         'male',
                                         'maxEducLevel_3',
                                         'maxEducLevel_4',
                                         'maxEducLevel_5',
                                         'maxEducLevel_6',
                                         'maxEducLevel_7',
                                         'maxEducLevel_nan',
                                         'college',
                                         'microEmpresa',
                                         'sizeFirm',
                                         'regSalud_2',
                                         'cotPension_3',
                                         'mes']])
x_test_model8 = poly.fit_transform(X_test[['age',
                                         'male',
                                         'maxEducLevel_3',
                                         'maxEducLevel_4',
                                         'maxEducLevel_5',
                                         'maxEducLevel_6',
                                         'maxEducLevel_7',
                                         'maxEducLevel_nan',
                                         'college',
                                         'microEmpresa',
                                         'sizeFirm',
                                         'regSalud_2',
                                         'cotPension_3',
                                         'mes']])

model8 =  LinearRegression().fit(x_train_model8,y_train)

In [ ]:
poly = PolynomialFeatures(degree=3)

x_train_model9 = poly.fit_transform(X_train)
x_test_model9 = poly.fit_transform(X_test)

model9 =  LinearRegression().fit(x_train_model9,y_train)

In [ ]:
poly = PolynomialFeatures(degree=4)

x_train_model10 = poly.fit_transform(X_train)
x_test_model10 = poly.fit_transform(X_test)

model10 =  LinearRegression().fit(x_train_model10,y_train)

In [ ]:
def compute_rmse(model, x_set, y_set):
  y_hat_model= model.predict(x_set)

  # Calculate Mean Squared Error
  mse_model = mean_squared_error(y_set, y_hat_model)

  # Calcular la raíz cuadrada del error cuadrático medio (RMSE)
  rmse_model = np.sqrt(mse_model)

  return rmse_model

In [ ]:
### Compute RMSE for each model
model_list = [model1, model2, model3, model4, model5, model6, model7, model8, model9, model10]
x_train_list  = [x_train_model1, x_train_model2, x_train_model3, x_train_model4, x_train_model5, x_train_model6, x_train_model7, x_train_model8, x_train_model9, x_train_model10]
x_test_list  = [x_test_model1, x_test_model2, x_test_model3, x_test_model4, x_test_model5, x_test_model6, x_test_model7, x_test_model8, x_test_model9, x_test_model10]

zipped = zip(x_train_list, x_test_list, model_list)
modelnames = [f"Model {i}" for i in range(1, len(model_list)+1)]

train_rmse = []
test_rmse = []
for x_train, x_test, model in zipped:
    train_rmse += [compute_rmse(model, x_train, y_train)]
    test_rmse += [compute_rmse(model, x_test, y_test)]

# Leave-one-out-cross-validation (LOOCV)

In [ ]:
kf = KFold(n_splits=10, shuffle=True, random_state=84)

In [ ]:
# Modelo
lin_reg = LinearRegression()
X1 = x_train_model5
X2 = x_train_model6
y = y_train

In [ ]:
scores = cross_val_score(model, X1, y, cv=kf, scoring='neg_mean_squared_error')

# Resultados
mse_scores = -scores  # Convertir a positivo

# Calcular la raíz cuadrada del error cuadrático medio (RMSE)
rmse_scores = np.sqrt(mse_scores)
model5_loocv = rmse_scores.mean()
print(f'RMSE por iteración: {rmse_scores}')
print(f'Media del RMSE: {model5_loocv}')

In [ ]:
scores = cross_val_score(model, X2, y, cv=kf, scoring='neg_mean_squared_error')

# Resultados
mse_scores = -scores  # Convertir a positivo

# Calcular la raíz cuadrada del error cuadrático medio (RMSE)
rmse_scores = np.sqrt(mse_scores)
model6_loocv = rmse_scores.mean()

print(f'RMSE por iteración: {rmse_scores}')
print(f'Media del RMSE: {model6_loocv}')

In [ ]:
# Plot errors
sns.set(rc={'figure.figsize':(8,6)})
sns.set_theme(style="ticks")
plt.plot(modelnames, train_rmse, label="Train RMSE")
plt.plot(modelnames, test_rmse, label="Test RMSE")
sns.despine()
plt.ylim([2200000,4000000])
plt.xticks(rotation=45)
plt.axhline(train_rmse[0], ls='--')
plt.scatter(["Model 5","Model 6"],[model5_loocv, model6_loocv], color="red", label="Test LOOCV RMSE")
plt.legend()
plt.savefig('views/RMSE_models.jpg',bbox_inches='tight', dpi=150)

## Análisis del modelo

In [ ]:
y_hat_model= model6.predict(x_test_model6)
y_diff = y_test - y_hat_model


fig = plt.scatter(y=y_diff, x=y_test, s=10)#, bw_adjust=0.15)
# plt.xlim([10,18])
# plt.ylim([10,18])
# ax.ticklabel_format(style='plain')
plt.gca().set_xlabel("Ingreso total observado")
plt.gca().set_ylabel("Error de estimación estimado (observado - estimado)")
sns.despine()
plt.savefig('views/ingreso_est_vs_obs.jpg',bbox_inches='tight', dpi=150)